# Langgraph with AgentNeo Integration
 This Jupyter notebook demonstrates the integration of AgentNeo, a powerful tracing and monitoring tool, with Langgraph, a graph-based approach to managing language models with an agent-based system to enhance the automation and decision-making capabilities of your application. This integration allows for comprehensive analysis and debugging of AI-powered systems.

 # Setup and Imports
First, let's import the necessary libraries and set up our environment.

In [ ]:
%%capture --no-stderr
%pip install -U langchain langchain_openai langsmith pandas langchain_experimental matplotlib langgraph langchain_core

In [ ]:

import os
import uuid
from dotenv import load_dotenv
from typing import List, Literal, Annotated
from typing_extensions import TypedDict
import openai

from langchain_core.messages import (
    SystemMessage, 
    HumanMessage, 
    AIMessage, 
    ToolMessage
)
from langchain_openai import ChatOpenAI
from pydantic import BaseModel

from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph.message import add_messages


# Load environment variables from .env file
load_dotenv()

# Initialize OpenAI API using environment variables
openai.api_key = os.getenv("OPENAI_API_KEY")



 # Initialize AgentNeo Session and Tracer
 Now, let's set up our AgentNeo session and tracer.



In [ ]:
from agentneo import AgentNeo, Tracer, Execution


# Initialize AgentNeo session
neo_session = AgentNeo(session_name="recipe_builder_assistant")
try:
    neo_session.create_project(project_name="Study_Guide")
except:
    neo_session.connect_project(project_name="Study_Guide")

# Create tracer
tracer = Tracer(session=neo_session)
tracer.start()


 # Define Agents and Tools
Now, let's create our AI tools using langgraph.

In [ ]:
# Define Agents and Tools

@tracer.trace_agent(name="StudyPlan")
class StudyPlan(BaseModel):
    """Study plan for data science topics."""
    topic: str
    duration: str
    key_concepts: List[str]

@tracer.trace_agent(name="StudyPlannerAgent")
class StudyPlannerAgent:
    def __init__(self):
        self.template = """You are a helpful data science study planner. Your job is to create structured study plans for various data science topics.

Given a data science topic:
1. Break down the topic into 3-5 key concepts or sub-topics
2. Suggest a realistic time allocation for each sub-topic
3. Recommend a total study duration"""

        self.llm = ChatOpenAI(temperature=0.7)
        self.llm_with_tool = self.llm.bind_tools([StudyPlan])

    @tracer.trace_tool(name="study_plan_chain")
    def study_plan_chain(self, state):
        messages = [SystemMessage(content=self.template)] + state["messages"]
        response = self.llm_with_tool.invoke(messages)
        return {"messages": [response]}


In [ ]:
@tracer.trace_agent(name="ConceptExplainerAgent")
class ConceptExplainerAgent:
    def __init__(self):
        self.template = """You are a data science concept explainer. Your job is to provide clear explanations of data science concepts and theories.

Given a data science concept:
1. Provide a concise definition
2. Explain the concept's importance in data science
3. Give a practical example or use case"""

        self.llm = ChatOpenAI(temperature=0.3)

    @tracer.trace_tool(name="concept_explanation_chain")
    def concept_explanation_chain(self, state):
        messages = [SystemMessage(content=self.template)] + state["messages"]
        response = self.llm.invoke(messages)
        return {"messages": [response]}



In [ ]:
@tracer.trace_agent(name="PracticalExerciseGenerator")
class PracticalExerciseGenerator:
    def __init__(self):
        self.template = """You are a practical exercise generator for data science topics. Your job is to create hands-on coding exercises or data analysis tasks.

Given a data science concept or topic:
1. Create a practical exercise that reinforces the concept
2. Provide a clear problem statement
3. Offer hints or tips for approaching the problem"""

        self.llm = ChatOpenAI(temperature=0.7)

    @tracer.trace_tool(name="generate_exercise_chain")
    def generate_exercise_chain(self, state):
        messages = [SystemMessage(content=self.template)] + state["messages"]
        response = self.llm.invoke(messages)
        return {"messages": [response]}

In [ ]:
@tracer.trace_agent(name="ResourceRecommenderAgent")
class ResourceRecommenderAgent:
    def __init__(self):
        self.template = """You are a resource recommender for data science topics. Your job is to suggest relevant learning materials.

Given a data science topic or concept:
1. Recommend 2-3 books (with authors) relevant to the topic
2. Suggest 1-2 online courses or MOOCs
3. Provide links to 2-3 high-quality tutorials or articles"""

        self.llm = ChatOpenAI(temperature=0.5)

    @tracer.trace_tool(name="recommend_resources_chain")
    def recommend_resources_chain(self, state):
        messages = [SystemMessage(content=self.template)] + state["messages"]
        response = self.llm.invoke(messages)
        return {"messages": [response]}

# Managing state graphs, saving memory, handling messages, and typing annotations.

In [ ]:
# State management

@tracer.trace_tool(name="get_state")
def get_state(state):
    messages = state["messages"]
    if isinstance(messages[-1], AIMessage) and messages[-1].tool_calls:
        return "add_tool_message"
    elif isinstance(messages[-1], AIMessage):
        if "study plan" in messages[-1].content.lower():
            return "concept_explanation"
        elif "concept" in messages[-1].content.lower():
            return "practical_exercise"
        elif "exercise" in messages[-1].content.lower():
            return "recommend_resources"
        else:
            return END
    elif not isinstance(messages[-1], HumanMessage):
        return END
    return "study_planning"

@tracer.trace_agent(name="State")
class State(TypedDict):
    messages: Annotated[list, add_messages]

# Initialize workflow
memory = MemorySaver()
workflow = StateGraph(State)

# Initialize agents
study_planner = StudyPlannerAgent()
concept_explainer = ConceptExplainerAgent()
exercise_generator = PracticalExerciseGenerator()
resource_recommender = ResourceRecommenderAgent()

# Add nodes to workflow
workflow.add_node("study_planning", study_planner.study_plan_chain)
workflow.add_node("concept_explanation", concept_explainer.concept_explanation_chain)
workflow.add_node("practical_exercise", exercise_generator.generate_exercise_chain)
workflow.add_node("recommend_resources", resource_recommender.recommend_resources_chain)

@workflow.add_node
def add_tool_message(state: State):
    return {
        "messages": [
            ToolMessage(
                content="Study plan generated!",
                tool_call_id=state["messages"][-1].tool_calls[0]["id"],
            )
        ]
    }

# Add edges to workflow
workflow.add_conditional_edges(
    "study_planning",
    get_state,
    ["add_tool_message", "concept_explanation", "practical_exercise", "recommend_resources", END]
)
workflow.add_edge("add_tool_message", "concept_explanation")
workflow.add_edge("concept_explanation", "practical_exercise")
workflow.add_edge("practical_exercise", "recommend_resources")
workflow.add_edge("recommend_resources", END)
workflow.add_edge(START, "study_planning")

# Compile graph
graph = workflow.compile(checkpointer=memory)

# Displaying a PNG image of a graph generated from a LangGraph instance using Mermaid visualization

In [ ]:
# %% 
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
# Main interaction loop
def run_interaction():
    config = {"configurable": {"thread_id": str(uuid.uuid4())}}

    print("Welcome to the Simplified Data Science Study Assistant!")
    print("You can ask for a study plan, concept explanations, practical exercises, or resource recommendations.")
    print("Type 'q' or 'quit' to exit.")

    while True:
        user = input("\nWhat would you like help with? ")
        
        if user.lower() in {"q", "quit"}:
            print("Thank you for using the Data Science Study Assistant! Good luck with your studies!")
            break
            
        output = None
        for output in graph.stream(
            {"messages": [HumanMessage(content=user)]}, 
            config=config, 
            stream_mode="updates"
        ):
            last_message = next(iter(output.values()))["messages"][-1]
            last_message.pretty_print()

        if output and "recommend_resources" in output:
            print("\nStudy plan, concept explanation, practical exercise, and resource recommendations completed!")
            print("Is there anything else you'd like to know?")



# Metrics Evaluation System

In [ ]:

# Metrics Evaluation System
def execute_metrics(neo_session, trace_id):
    """Execute and return metrics analysis."""
    exe = Execution(session=neo_session, trace_id=trace_id)
    exe.execute(metric_list=[
        'goal_decomposition_efficiency',
        'goal_fulfillment_rate',
        'tool_call_success_rate_metric'
    ])
    return exe.get_results()

def print_metrics_results(metric_results):
    """Print the metrics results in a formatted way."""
    print("\nMetrics Results:")
    print(metric_results)

def launch_metrics_dashboard(neo_session):
    """Launch the AgentNeo metrics dashboard."""
    neo_session.launch_dashboard()

In [ ]:
if __name__ == "__main__":
    try:
        run_interaction()
    finally:
        tracer.stop()
        print(f"Trace ID: {tracer.trace_id}")
        


In [ ]:
results = execute_metrics(neo_session, tracer.trace_id)


In [ ]:
# Display results
print_metrics_results(results)
        


In [ ]:
results

In [ ]:
# Launch dashboard
launch_metrics_dashboard(neo_session)